In [92]:
import os
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim

from torchvision import transforms, models
from torchvision.transforms import Resize, ToTensor, Normalize
import import_ipynb
import numpy as np

from DataLoader import loader_train, loader_val

from efficientnet_pytorch import EfficientNet

In [93]:
# len_valid_set = int(0.3 * len(train_dataset))
# len_train_set = len(train_dataset) - len_valid_set
# print(len_valid_set, len_train_set)

In [94]:
#train_data, valid_data = torch.utils.data.random_split(train_dataset, [len_train_set, len_valid_set])

In [95]:
# loader_train = DataLoader(train_data, batch_size = 4, shuffle = True)
# loader_valid = DataLoader(valid_data, batch_size = 4, shuffle = False)

In [96]:
device = torch.device('cuda')
#resnet18 = models.resnet18(pretrained = True)
#resnet18.fc = nn.Linear(in_features=512, out_features=18, bias=True)

In [97]:
#resnet152 = models.resnet152(pretrained = True)
#resnet152.fc = nn.Linear(in_features=2048, out_features=18, bias=True)

In [98]:
vgg16 = models.vgg16(pretrained=True)
#vgg16.classifier[6] = nn.Linear(in_features=4096, out_features=18, bias=True)
vgg16.classifier = nn.Sequential(
    nn.Linear(512 * 7 * 7, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(True),
    nn.Dropout(),
    nn.Linear(4096, 18),
)


# Freeze only feature parts
#vgg16.features.requires_grad_(False)
#for param, weight in vgg16.named_parameters():
#    print(f"param {param:20} required gradient? -> {weight.requires_grad}")


In [99]:
#vgg19 = models.vgg19(pretrained=True)
#vgg19.classifier[6] = nn.Linear(in_features=4096, out_features=18, bias=True)

In [100]:
# efficient = EfficientNet.from_pretrained('efficientnet-b0')
# efficient._fc = nn.Linear(in_features=1280, out_features=18, bias=True)
#efficient

In [101]:
"""
Initialize all weight using xavier uniform.
For more weight initialization methods, check https://pytorch.org/docs/stable/nn.init.html  
""" 
import torch.nn.init as init

def initialize_weights(model):
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            init.xavier_uniform_(m.weight.data)
            if m.bias is not None:
                m.bias.data.zero_()
        elif isinstance(m, nn.BatchNorm2d):
            m.weight.data.fill_(1)
            m.bias.data.zero_()
        elif isinstance(m, nn.Linear):
            m.weight.data.normal_(0, 0.01)
            m.bias.data.zero_()
            
#initialize_weights(vgg16.classifier) #classifier만 초기화

In [102]:
#criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(resnet18.parameters(), lr=0.01)#optim.Adam(model.parameters(), lr=0.001)

In [103]:
class Trainer:
    def __init__(self, model, optimizer, device, t_loader, v_loader):
        self.device = device
        self.t_loader = t_loader
        self.v_loader = v_loader
        self.model = model.to(device)
        self.criterion = nn.CrossEntropyLoss()
        
        update_params = [p for p in model.parameters() if p.requires_grad]
        self.optimizer = optimizer(update_params, lr= 0.0001, momentum = 0.9, weight_decay = 0.1)
        
    def train_eval(self):
        print("Training Start")
        for epoch in range(5):
            print(epoch+1)
            self.train_one_epoch(epoch)
            self.test_one_epoch(epoch)
    
    def train_one_epoch(self,epoch):
        self.model.train()

        running_loss = 0.0
        for i, (inputs, targets) in enumerate(self.t_loader):
            inputs, targets = inputs.to(self.device), targets.to(self.device)

            self.optimizer.zero_grad()

            outputs = self.model(inputs)

            loss = self.criterion(outputs, torch.max(targets,1)[1])

            loss.backward()
            self.optimizer.step()

            running_loss += loss.item()

            if i % 1000 == 0:
                print(f"epoch : {epoch+1}  train Loss : {running_loss / (i+1)}")
                running_loss = 0.0
                
                
    def test_one_epoch(self,epoch):
        self.model.eval()
        test_loss = 0.0
        
        with torch.no_grad():
            for i , (inputs, targets) in enumerate(self.v_loader):
                inputs, targets = inputs.to(self.device), targets.to(self.device)
                outputs = self.model(inputs)
                loss = self.criterion(outputs, torch.max(targets,1)[1])
                
                test_loss += loss.item()
                
                if i % 1000 == 0:
                    print(f"epoch : {epoch+1}  test Loss : {test_loss / (i+1)}")
                    test_loss = 0.0

In [104]:
optimizer = optim.__dict__['SGD']
trainer = Trainer(vgg16, optimizer, device, loader_train, loader_val)

In [105]:
trainer.train_eval()

Training Start
1
epoch : 1  train Loss : 2.8396284580230713
epoch : 1  train Loss : 1.3470734477426474
epoch : 1  train Loss : 0.30299919056727076
epoch : 1  train Loss : 0.17747228241052945
epoch : 1  test Loss : 0.22437798976898193
epoch : 1  test Loss : 0.5323929862750993
2
epoch : 2  train Loss : 1.7192609310150146
epoch : 2  train Loss : 0.4452851424996204
epoch : 2  train Loss : 0.24279298584420478
epoch : 2  train Loss : 0.16947738827857592
epoch : 2  test Loss : 0.37056922912597656
epoch : 2  test Loss : 0.45154004432372635
3
epoch : 3  train Loss : 1.2807950973510742
epoch : 3  train Loss : 0.543284729093343
epoch : 3  train Loss : 0.27375950347231903
epoch : 3  train Loss : 0.20802555943447296
epoch : 3  test Loss : 0.34436285495758057
epoch : 3  test Loss : 0.6120617620285962
4
epoch : 4  train Loss : 0.2749505937099457
epoch : 4  train Loss : 0.6340903137642128
epoch : 4  train Loss : 0.34723422503138873
epoch : 4  train Loss : 0.24023833083810012
epoch : 4  test Loss : 0.5

In [37]:
# resnet18.train()
# for epoch in range(20):
#     running_loss = 0.0
#     for i, data in enumerate(train_loader,0):
#         inputs, label = data
#         inputs = inputs.to(device)
#         label = label.to(device)
        
#         optimizer.zero_grad()
        
#         pred = resnet18(inputs)
#         loss = criterion(pred, torch.max(label,1)[1])
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item()
#         #print(running_loss)
#         if i % 1000 == 999:
#             print(f"epoch : {epoch+1} {i+1} loss : {running_loss / 1000}")
#             running_loss = 0.0
# print("Finishing Training")

In [ ]:
class TestDataset(Dataset):
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform
     
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])

        if self.transform:
            image = self.transform(image)
            
        return image

    def __len__(self):
        return len(self.img_paths)

In [ ]:
class albumentation_TestDataset(Dataset):
     
    
    def __init__(self, img_paths, transform):
        self.img_paths = img_paths
        self.transform = transform
        
    def __getitem__(self, index):
        image = Image.open(self.img_paths[index])
    
        if self.transform:
            image_transform = self.transform(image=np.array(image))['image']
        
        return image_transform
    
    def __len__(self):
        return len(self.img_paths)

In [ ]:
def albumentation_transform():
    import albumentations as A
    return A.Compose([
        Resize(224,224),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
        A.pytorch.ToTensorV2(p=1.0),
    ])
#albumentation_transform = albumentation_transform()

In [ ]:
# meta 데이터와 이미지 경로를 불러옵니다.
test_dir = '/opt/ml/input/data/eval'
submission = pd.read_csv(os.path.join(test_dir, 'info.csv'))
image_dir = os.path.join(test_dir, 'images')

# Test Dataset 클래스 객체를 생성하고 DataLoader를 만듭니다.
image_paths = [os.path.join(image_dir, img_id) for img_id in submission.ImageID]
# transform_al = Compose([
#     Resize(224, 224), # Image.BILINEAR 을 적용하면 albumentaion이 적용이 안되는거 같다
#     Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225), max_pixel_value=255.0, p=1.0),
#     ToTensorV2(p=1.0),
# ], p = 1.0)

transform = transforms.Compose([
    Resize((224, 224), Image.BILINEAR),
    ToTensor(),
    Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
])

dataset = TestDataset(image_paths, transform)

loader = DataLoader(
    dataset,
    shuffle=False
)

In [ ]:
#albumentation_dataset = albumentation_TestDataset(image_paths, albumentation_transform)
#albumentation_loader = DataLoader(albumentation_dataset, shuffle=False)

In [ ]:
def make_submission(data_loader, model, t):
    model.eval()
    test_dir = '/opt/ml/input/data/eval'
    
    # 모델이 테스트 데이터셋을 예측하고 결과를 저장합니다.
    all_predictions = []
    for images in loader:
        with torch.no_grad():
            images = images.to(device)
            pred = model(images)
            pred = pred.argmax(dim=-1)
            all_predictions.extend(pred.cpu().numpy())
    submission['ans'] = all_predictions
    sub_name = 'submission' + str(t) + '.csv'
    # 제출할 파일을 저장합니다.
    submission.to_csv(os.path.join(test_dir, sub_name), index=False)
    print('test inference is done!')

In [ ]:
make_submission(loader, vgg16, 22)

In [67]:
#model 저장하기
def save_model(model, name):
    save_folder = '/opt/ml/input/data/eval/model'
    save_path = os.path.join(save_folder, name + ".pth")
    
    torch.save(model.state_dict(), save_path)
    print(f"Model saving success at {save_path}")
    
save_model(efficient, "sub21_vgg16")

Model saving success at /opt/ml/input/data/eval/model/sub21_vgg16.pth
